In [1]:
import numpy as np
import os
import pandas as pd
import astropy.coordinates as coord
from tqdm import tqdm

In [7]:
PATH = os.path.normpath(os.getcwd() + os.sep + os.pardir)
DATAPATHmaster = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data'
CODEPATH = os.path.join(PATH,'Code')
DATAPATH = os.path.join(DATAPATHmaster,'Sjoert_Flares')
print(DATAPATH)
columns = ['sindex', 'field', 'ccdid', 'qid', 'filter', 'pid', 'infobitssci', 'sciinpseeing', 'scibckgnd', 'scisigpix', 'zpmaginpsci', 'zpmaginpsciunc', 'zpmaginpscirms', 'clrcoeff', 'clrcoeffunc', 'ncalmatches', 'exptime', 'adpctdif1', 'adpctdif2', 'diffmaglim', 'zpdiff', 'programid', 'jd', 'rfid', 'forcediffimflux', 'forcediffimfluxunc', 'forcediffimsnr', 'forcediffimchisq', 'forcediffimfluxap', 'forcediffimfluxuncap', 'forcediffimsnrap', 'aperturecorr', 'dnearestrefsrc', 'nearestrefmag', 'nearestrefmagunc', 'nearestrefchi', 'nearestrefsharp', 'refjdstart', 'refjdend', 'procstatus']
dtypes = [(columns[x],float) for x in range(len(columns))]
dtypes[4] = ('filter',r'U8')

C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\Sjoert_Flares


In [4]:

test = list(open(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\18\ZTF18aasuray\batchfp_req0000317597_lc.txt','r'))

simeon_data = pd.read_csv(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\dump\all_nuclear_transients.csv').dropna()
simeon_data.reset_index(drop=True,inplace=True)
simeon_data.shape

(14245, 3)

#### Two cells below take the full ZTF queried wget command list and takes out the ones that are from Sjoert's paper - these are not in Simeon's list (I think)

In [4]:
with open(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\wget.txt','r') as wget:
    wget_commands = wget.readlines()

wget_commands_files = [line[55:][:28] for line in wget_commands]

lines_to_delete = []
for file in os.listdir(DATAPATH):
    if file in wget_commands_files:
        i = np.where(np.array(wget_commands_files) == file)[0][0]
        lines_to_delete.append(wget_commands[i].strip("\n"))

lines_to_delete

[]

In [5]:
with open(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\wget.txt', "r") as f:
    lines = f.readlines()
with open(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\wget.txt', "w") as f:
    for line in lines:
        if line.strip("\n") not in lines_to_delete:
            f.write(line)

In [5]:
catalog_coords = coord.SkyCoord(np.array(simeon_data['RA']),np.array(simeon_data['Dec']),unit='deg')

def get_year(file,catalog_coords=catalog_coords):
    #get data from file into a list and close it again
    with  open(file,'r') as data:
        data_list = data.readlines()

    #get the relevant lines from the list
    ra_line = data_list[3]
    dec_line = data_list[4]

    ra = float(ra_line[25:][:-9])
    dec = float(dec_line[25:][:-9]) #the first 26 and the last 10 characters can be deleted from the line to get only the numerical value for both RA en Dec in all files
    matchcoord = coord.SkyCoord(ra,dec,frame='icrs',unit='deg')
    idx,_,_ = coord.match_coordinates_sky(matchcoord,catalog_coords)
    idx = int(idx)

    ID = simeon_data._get_value(idx,'ztf_id') #get the ZTF id of the simeon data file entry with least angular separation
    year = int(ID[3:5]) #distill the year from the ID
    return year,ID


In [ ]:
wget = open(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\wget.txt','r')

for line in tqdm(wget):
    #Boolean that tracks if the current wget is going to download a file that already exists according to the log file.
    skip_iter = False

    #extract the file name from the wget command and note its current path in the DOWNLOADED folder
    filename = line[55:][:28]
    filepath = os.path.join(down_path,filename)
all_ids = []

In [10]:
#change this to wget.txt instead of wget_test.txt in order to run on ALL lightcurves except for Sjoert's.
# wget = open(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\wget_test.txt','r')
wget = open(r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\dump\wget.txt','r')
downloaded = open(r"C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\dump\downloaded_files.txt",'a+')

#maybe a truncated view? seperate into multiple functions (since the log writing code is repeated twice now for instance). This is just
#to make it more pretty not any functionality

down_path = os.path.join(DATAPATHmaster,"DOWNLOADED")

def wget_move_lightcurve(wget_file,log_file,down_path,verbose=True):
    """Downloads lightcurves per wget commands that are in seperate lines in a .txt file to the "down_path" directory 
       and moves them to the correct directory for oversightful bookkeeping.
       Is able to handle duplicates; if a downloaded file already exists at the right location the new download is deleted.
       Logs the downloads and subsequent directories in the a log file ("log_file"). New downloads only logged only if the 
       file doesn't already exist in its right location.

    Args:
        wget_file (string): Full path to the .txt file that holds the wget commands to be executed.
        log_file (string): Full path to the log_file (.txt) which holds the filename and correct path of all downloaded lightcurves. This file should never be manually touched.
        down_path (string): Full path to the directory where the wget commands will be executed and thus the files be downloaded before being moved.
        verbose (boolean): Boolen value that, for now, only silences the prompt when a file is already downloaded (since these may take up a lot of space) when set to False.
                           Default is True.
    """
    #proceed only if there's not already some files in DOWNLOADED, else there might be duplicates.
    if len(os.listdir(down_path)) == 0:
        for line in tqdm(wget_file):
            #Boolean that tracks if the current wget is going to download a file that already exists according to the log file.
            skip_iter = False

            #extract the file name from the wget command and note its current path in the DOWNLOADED folder
            filename = line[55:][:28]
            filepath = os.path.join(down_path,filename)

            #Place to store empty lightcurves (still stored for possible further inspection.)
            emptypath = r'C:\Users\timvd\Documents\Uni 2023-2024\First Research Project\Data\EMPTY'

            #if the file to be downloaded (as extracted from the wget command) is already in the download log_file, skip the iteration after verifying the log file 
            # and continue to the next wget command before downloading.
            log_file.seek(0)
            flag = True
            for l in log_file.readlines():
                if filename in l:
                    already_downloaded_path = l[29:].strip("\n")
                    if verbose:
                        print(f'Download log shows {filename} is already downloaded at {already_downloaded_path}. Continuing to next command.')
                    skip_iter = True
                    break
                else:
                    if flag:
                        print(filename)
                        flag = False

            if skip_iter:
                #check if the log file is even correct; see if the file to be downloaded actually exists in the path specified by the log file.
                #if it isn't continue the download and move it to the correct place. The log file will then be correct as well - it doesn't need manual correction.
                if filename in os.listdir(already_downloaded_path):
                    continue
                else:
                    print(f"Log file was incorrect for file {filename}, suspected manual deletion of lightcurve. Continuing download.")

            #change to the download directory
            os.chdir(down_path)
            os.system(line)

            #get year and transient (ZTF) ID from the lightcurve file
            try:
                year,ID = get_year(filepath)
                yearpath = os.path.join(DATAPATHmaster,str(year))
                savepath = os.path.join(yearpath,ID)
            #This is triggered when the file is empty
            except IndexError:
                # if verbose:
                log_string = f'{filename} {emptypath}\n'
                print("Downloaded lightcurve is empty. Moving to empty folder.")

                try:
                    os.rename(filepath,os.path.join(emptypath,filename))
                    #if skip_iter is set to True, then the log_file will already have an entry somewhere specifying the file in the current iteration which we can trust to be true. 
                    # Then the log file needn't be updated.
                    if not skip_iter:
                        #if the the exact log_string is not yet in the log file, write it to the log file.
                        if log_string not in log_file.readlines():
                            log_file.write(log_string)

                except FileExistsError:
                    print(f"There already is a file with this exact name ({filename}) at the specified location. Deleting the new download and updating log file.")
                    os.remove(filepath)
                    #if it so happens that a file is not logged but it is found now that it does in fact exist, update the log file.
                    if not skip_iter:
                        #if the the exact log_string is not yet in the log file, write it to the log file.
                        if log_string not in log_file.readlines():
                            log_file.write(log_string)
                
                #and continue to the next download
                continue


            #if a folder for the lightcurve does not already exist, make it.
            if ID not in os.listdir(yearpath): 
                os.mkdir(savepath)

            #move the file from the download folder to its rightful location
            #if the file already exists in its right location, let the user know and delete the newly downloaded file 
            # this is because the lightcurves shouldn't be any different or better.
            #Moreover, don't log the file in the log_file if the file already exists in its location - it should then already be in the log_file.
            log_string = f'{filename} {savepath}\n'
            try:
                os.rename(filepath,os.path.join(savepath,filename))
                #if skip_iter is set to True, then the log_file will already have an entry somewhere specifying the file in the current iteration which we can trust to be true. 
                # Then the log file needn't be updated.
                if not skip_iter:
                    #if the the exact log_string is not yet in the log file, write it to the log file.
                    if log_string not in log_file.readlines():
                        log_file.write(log_string)

            except FileExistsError:
                print(f"There already is a file with this exact name ({filename}) at the specified location. Deleting the new download and updating log file.")
                os.remove(filepath)
                #if it so happens that a file is not logged but it is found now that it does in fact exist, update the log file.
                if not skip_iter:
                    #if the the exact log_string is not yet in the log file, write it to the log file.
                    if log_string not in log_file.readlines():
                        log_file.write(log_string)

    else:
        del_down = input(f'There are already {len(os.listdir(down_path))} files in the "DOWNLOADED" folder, would you like to delete these? y/n ')
        if del_down.lower() == 'y':
            for file in os.listdir(down_path):
                os.remove(file)
        else:
            print("Please remove all files from the directory before proceeding.")

    #close files and return the working directory to the one at the start of the code.        
    wget_file.close()
    downloaded.close()
    os.chdir(PATH)

wget_move_lightcurve(wget,downloaded,down_path,verbose=False)

5it [00:00, 47.91it/s]

batchfp_req0000317596_lc.txt
batchfp_req0000317597_lc.txt
batchfp_req0000317598_lc.txt
batchfp_req0000317599_lc.txt
batchfp_req0000317600_lc.txt
batchfp_req0000317601_lc.txt
batchfp_req0000317602_lc.txt
batchfp_req0000317603_lc.txt
batchfp_req0000317604_lc.txt
batchfp_req0000317605_lc.txt


19it [00:00, 63.46it/s]

batchfp_req0000317606_lc.txt
batchfp_req0000317607_lc.txt
batchfp_req0000317608_lc.txt
batchfp_req0000317609_lc.txt
batchfp_req0000317610_lc.txt
batchfp_req0000317611_lc.txt
batchfp_req0000317612_lc.txt
batchfp_req0000317613_lc.txt
batchfp_req0000317614_lc.txt
batchfp_req0000317615_lc.txt
batchfp_req0000317616_lc.txt
batchfp_req0000317617_lc.txt
batchfp_req0000317618_lc.txt
batchfp_req0000317619_lc.txt
batchfp_req0000317620_lc.txt


40it [00:00, 63.87it/s]

...

692it [00:08, 80.55it/s]

batchfp_req0000359313_lc.txt
batchfp_req0000359314_lc.txt
batchfp_req0000359315_lc.txt
batchfp_req0000359316_lc.txt
batchfp_req0000359317_lc.txt
batchfp_req0000359318_lc.txt
batchfp_req0000359319_lc.txt
batchfp_req0000359320_lc.txt
batchfp_req0000359321_lc.txt
batchfp_req0000359322_lc.txt
batchfp_req0000359323_lc.txt
batchfp_req0000359324_lc.txt
batchfp_req0000359325_lc.txt
batchfp_req0000359326_lc.txt
batchfp_req0000359327_lc.txt
batchfp_req0000359328_lc.txt
batchfp_req0000359329_lc.txt


701it [00:08, 78.05it/s]

batchfp_req0000359330_lc.txt
batchfp_req0000359331_lc.txt
batchfp_req0000359332_lc.txt
batchfp_req0000359333_lc.txt
batchfp_req0000359334_lc.txt


KeyboardInterrupt: 